# Querying chpaf MoPo

In [1]:
import rdflib
import pandas as pd

# Set pandas to display wide columns
pd.set_option('max_colwidth', 1000)

g = rdflib.Graph()
g.parse("../examples/mopo_mvp.ttl")

<Graph identifier=N74b76bf671e545bfb42f26cd573ef2b4 (<class 'rdflib.graph.Graph'>)>

## Chapter 1 - Proposal for Abandonment

In [2]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?text ?proposal WHERE {

    ?ProposalEntity chpaf:proceduralRequestProposal ?proposal;
        ^prov:used ?ProposalActivity.

    ?ProposalActivity prov:used/chpaf:parliamentaryAffairIdentifier ?id.
    ?ProposalActivity prov:used/chpaf:proceduralRequestTitle ?title.
    ?ProposalActivity prov:used/chpaf:proceduralRequestText ?text.

} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df
# ~0.3 s

,id,title,text,proposal
0,06.3009,Einheitliche Finanzierung von Spital- und ambulanten Leistungen,"Der Bundesrat wird beauftragt, dem Parlament bis Ende 2008 eine Vorlage für eine einheitliche Finanzierung von Spital- und ambulanten Leistungen auf der Grundlage der Vorlage der SGK-SR vorzulegen. Dabei soll auch geprüft werden, ob und mit welchen Kompetenzen und Konsequenzen eine Zahl- oder Clearingstelle eingeführt werden soll.",Die Motionen stehen im Zusammenhang mit der Beratung der parlamentarischen Initiative Humbel vom 11. Dezember 2009 (09.528 «Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus»). In der Beratung dieser parlamentarischen Initiative wurden die angesprochenen Themen von den eidgenössischen Räten aufgenommen und eingehend behandelt. Aufgrund dieser Beratungen und diverser Berichte und Vorschläge von Seiten der Bundesverwaltung wurde eine Änderung des Bundesgesetzes über die Krankenversicherung (SR 832.10) erarbeitet. Am 22. Dezember 2023 haben die eidgenössischen Räte die Gesetzesänderung verabschiedet (BBl 2024 31; Einheitliche Finanzierung der Leistungen). \nDer Bundesrat erachtet die Anliegen der Motionen als erfüllt und beantragt deren Abschreibung. \n\n\n\n\n\n\n
1,09.3535,Leistungsfinanzierung nach dem KVG vereinheitlichen,"Der Bundesrat wird beauftragt, die Finanzierung der Leistungen nach dem Bundesgesetz über die Krankenversicherung (KVG) zu vereinheitlichen. Für den stationären und den ambulanten Bereich sollen die gleichen Finanzierungsgrundsätze und der gleiche Verteilschlüssel gelten.",Die Motionen stehen im Zusammenhang mit der Beratung der parlamentarischen Initiative Humbel vom 11. Dezember 2009 (09.528 «Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus»). In der Beratung dieser parlamentarischen Initiative wurden die angesprochenen Themen von den eidgenössischen Räten aufgenommen und eingehend behandelt. Aufgrund dieser Beratungen und diverser Berichte und Vorschläge von Seiten der Bundesverwaltung wurde eine Änderung des Bundesgesetzes über die Krankenversicherung (SR 832.10) erarbeitet. Am 22. Dezember 2023 haben die eidgenössischen Räte die Gesetzesänderung verabschiedet (BBl 2024 31; Einheitliche Finanzierung der Leistungen). \nDer Bundesrat erachtet die Anliegen der Motionen als erfüllt und beantragt deren Abschreibung. \n\n\n\n\n\n\n
2,09.3546,Transparente Finanzierung der sozialen Grundversicherung,"Der Bundesrat wird beauftragt, ein neues Finanzierungssystem für die Grundversicherung vorzuschlagen, welches:\n- Transparenz in Bezug auf die Finanzierung der Grundversicherung herstellt;\n- die heutigen falschen Anreize zwischen ambulantem und stationärem Bereich beseitigt;\n- prozentual festlegt, welche Kosten über Prämien der Versicherten und welche über Steuern zu finanzieren sind. In diesem Zusammenhang ist die Aufteilung des steuerfinanzierten Anteils zwischen Bund und Kantonen ebenfalls zu definieren.",Die Motionen stehen im Zusammenhang mit der Beratung der parlamentarischen Initiative Humbel vom 11. Dezember 2009 (09.528 «Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus»). In der Beratung dieser parlamentarischen Initiative wurden die angesprochenen Themen von den eidgenössischen Räten aufgenommen und eingehend behandelt. Aufgrund dieser Beratungen und diverser Berichte und Vorschläge von Seiten der Bundesverwaltung wurde eine Änderung des Bundesgesetzes über die Krankenversicherung (SR 832.10) erarbeitet. Am 22. Dezember 2023 haben die eidgenössischen Räte die Gesetzesänderung verabschiedet (BBl 2024 31; Einheitliche Finanzierung der Leistungen). \nDer Bundesrat erachtet die Anliegen der Motionen als erfüllt und beantragt deren Abschreibung. \n\n\n\n\n\n\n
3,09.3089,Vertriebsanteil bei den Medikamentenkosten,"Der Bundesrat wird beauftragt, für den Vertrieb von Arzneimitteln eine preisunabhängige Marge, differenziert nach Vertriebskanal, festzulegen. Diese soll auf der Basis einer effizienten 

## Chapter 2 - Information on the Fulfilment Status

In [3]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?text ?information WHERE {

    ?InformationEntity chpaf:proceduralRequestInformation ?information;
        ^prov:used ?InformationActivity.

    ?InformationActivity prov:used/chpaf:parliamentaryAffairIdentifier ?id.
    ?InformationActivity prov:used/chpaf:proceduralRequestTitle ?title.
    ?InformationActivity prov:used/chpaf:proceduralRequestText ?text.

} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df
# ~0.3 s

,id,title,text,information
0,04.3276,Übergang zur Individualbesteuerung,"Der Bundesrat wird beauftragt, dem Parlament so rasch wie möglich unter Einbezug der Kantone einen Gesetzentwurf vorzulegen, welcher den Übergang von der Ehepaar- und Familienbesteuerung zur zivilstandsunabhängigen Individualbesteuerung vorsieht. Dabei darf die steuerliche Belastung für den einzelnen Steuerpflichtigen das aktuelle Niveau nicht überschreiten.","Abschreibung beantragt mit Geschäft 18.034: Botschaft vom 21. März 2018 zur Änderung des Bundesgesetzes über die direkte Bundessteuer (Ausgewogene Paar- und Familienbesteuerung). Am 18. Dezember 2019 wies das Parlament das Geschäft an den Bundesrat zurück mit dem Auftrag, alternative Modelle vorzulegen. Zudem beschloss das Parlament in der Herbstsession 2020, die Verabschiedung einer Botschaft zur Einführung der Individualbesteuerung in die Legislaturplanung 2019-2023 aufzunehmen. Am 2. Dezember 2022 eröffnete der Bundesrat die Vernehmlassung zum Bundesgesetz über die Individualbesteuerung. Diese dauerte bis zum 16. März 2023. Am 30. August 2023 hat der Bundesrat die Eckwerte für die Botschaft zur Individualbesteuerung festgelegt. Die Botschaft zur Steuergerechtigkeits-Initiative und zum indirekten Gegenvorschlag wird voraussichtlich im ersten Quartal 2024 verabschiedet werden."
1,05.3299,Übergang zur Individualbesteuerung. Dringliche Massnahmen,"Bei der Besteuerung der natürlichen Personen sind rasch Massnahmen zu ergreifen, die der Rechtsprechung zur Gleichbehandlung verheirateter und unverheirateter Paare Rechnung tragen und den Wechsel zur Individualbesteuerung nicht verhindern.","Abschreibung beantragt mit Geschäft 18.034: Botschaft vom 21. März 2018 zur Änderung des Bundesgesetzes über die direkte Bundessteuer (Ausgewogene Paar- und Familienbesteuerung). Am 18. Dezember 2019 wies das Parlament das Geschäft an den Bundesrat zurück mit dem Auftrag, alternative Modelle vorzulegen. Zudem beschloss das Parlament in der Herbstsession 2020, die Verabschiedung einer Botschaft zur Einführung der Individualbesteuerung in die Legislaturplanung 2019-2023 aufzunehmen. Am 2. Dezember 2022 eröffnete der Bundesrat die Vernehmlassung zum Bundesgesetz über die Individualbesteuerung. Diese dauerte bis zum 16. März 2023. Am 30. August 2023 hat der Bundesrat die Eckwerte für die Botschaft zur Individualbesteuerung festgelegt. Die Botschaft zur Steuergerechtigkeits-Initiative und zum indirekten Gegenvorschlag wird voraussichtlich im ersten Quartal 2024 verabschiedet werden."
2,10.4127,Beseitigung der Heiratsstrafe,"Der Bundesrat wird beauftragt, die notwendigen Gesetzesrevisionen vorzulegen, damit die heutige Benachteiligung von Verheirateten gegenüber Konkubinatspaaren und Alleinstehenden sowie von Einverdiener- gegenüber Doppelverdienerehepaaren im Steuerrecht auf Bundes- und Kantonsebene beseitigt wird.","Abschreibung beantragt mit Geschäft 18.034: Botschaft vom 21. März 2018 zur Änderung des Bundesgesetzes über die direkte Bundessteuer (Ausgewogene Paar- und Familienbesteuerung). Am 18. Dezember 2019 wies das Parlament das Geschäft an den Bundesrat zurück mit dem Auftrag, alternative Modelle vorzulegen. Zudem beschloss das Parlament in der Herbstsession 2020, die Verabschiedung einer Botschaft zur Einführung der Individualbesteuerung in die Legislaturplanung 2019-2023 aufzunehmen. Am 2. Dezember 2022 eröffnete der Bundesrat die Vernehmlassung zum Bundesgesetz über die Individualbesteuerung. Diese dauerte bis zum 16. März 2023. Am 30. August 2023 hat der Bundesrat die Eckwerte für die Botschaft zur Individualbesteuerung festgelegt. Die Botschaft zur Steuergerechtigkeits-Initiative und zum indirekten Gegenvorschlag wird voraussichtlich im ersten Quartal 2024 verabschiedet werden."
3,11.3545,Zivilstandsunabhängige Besteuerung und Renten,"Der Bundesrat wird beauftragt, Lösungen zu prüfen, die eine weitgehend zivilstandsunabhängige Besteuerung sowie zivilstandsunabhängige Renten ermöglichen.","Abschreibung beantragt

## Connex Procedural Requests

Needs the class `chpaf:ConnexActivity` being added to the corresponding activity:

In [4]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?main ?dep WHERE {

    ?depActivity a chpaf:ProceduralRequestConnexActivity;
        prov:used/chpaf:parliamentaryAffairIdentifier ?dep.
    
    ?depActivity prov:used ?entity.

    ?mainActivity prov:used ?entity;
        prov:used/chpaf:parliamentaryAffairIdentifier ?main.

    FILTER NOT EXISTS {?mainActivity a chpaf:ProceduralRequestConnexActivity}

} ORDER BY ?main

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,main,dep
0,04.3276,05.3299
1,04.3276,10.4127
2,04.3276,11.3545
3,04.3276,14.3005
4,04.3276,16.3044
...,...,...
124,21.3234,21.3457
125,21.3290,21.4342
126,21.3450,21.4531
127,21.3877,21.3968


## Deleting all Triples of a Single Report

In [5]:
# SPARQL DELETE query
delete_query = """
    PREFIX chpaf: <https://ch.paf.link/>
    
    DELETE {
        ?s ?p ?o .
    }
    WHERE {
        ?s a ?class.
        VALUES ?class { 
            chpaf:ParliamentaryAffairIdentifierEntity
            chpaf:ProceduralRequestEntity
            chpaf:ProceduralRequestProposalEntity 
            chpaf:ProceduralRequestProposalActivity 
            chpaf:ProceduralRequestInformationEntity 
            chpaf:ProceduralRequestInformationActivity 
            chpaf:ProceduralRequestReportEntity 
            chpaf:ProceduralRequestReportActivity
        }.
        ?s ?p ?o.
    }
"""

# Execute the DELETE query
g.update(delete_query)

# Check the result
query_string = """


SELECT ?s ?p ?o WHERE {

    ?s ?p ?o.

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,s,p,o
0,https://ch.paf.link/Postulate,https://ch.paf.link/parlApiId,6
1,https://ch.paf.link/Postulate,https://ch.paf.link/termdat,https://register.ld.admin.ch/termdat/109124
2,https://ch.paf.link/Motion,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://ch.paf.link/ProceduralRequestType
3,https://ch.paf.link/Postulate,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://ch.paf.link/ProceduralRequestType
4,https://ch.paf.link/Motion,https://ch.paf.link/termdat,https://register.ld.admin.ch/termdat/109123
5,https://ch.paf.link/Motion,https://ch.paf.link/parlApiId,5
